# 데이터 불러오기

In [9]:
import glob
import os, re 
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus에 담기
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['[Verse 1]', 'They come from everywhere', 'A longing to be free']


# 전처리 과정 preprocessing
텍스트 분류 모델에서 많이 본 것처럼 텍스트 생성 모델에도 단어 사전을 만들게 되며, 문장을 일정한 기준으로 쪼개게 된다. 그 과정을 토큰화(Tokenize) 라고 합니다.

In [10]:
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
# 이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있겠네요!
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

In [68]:
corpus = []
# preprocessing
for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뛰기
    if len(sentence) == 0 : continue
    if sentence[-1] == ":": continue
    
    # 정제 후 corpus에 추가하기
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보기
print(corpus[:10])

['<start> verse <end>', '<start> they come from everywhere <end>', '<start> a longing to be free <end>', '<start> they come to join us here <end>', '<start> from sea to shining sea and they all have a dream <end>', '<start> as people always will <end>', '<start> to be safe and warm <end>', '<start> in that shining city on the hill some wanna slam the door <end>', '<start> instead of opening the gate <end>', '<start> aw , let s turn this thing around <end>']


In [69]:
# tokenize
def tokenize(corpus):
    # 12000단어를 기억할 수 있는 tokenizer를 만든다.
    # 우리는 이미 문장을 정제했으니 filters는 설정할 필요가 없다
    # 12000단어에 포함되지 못한 단어는 '<unk>'로 바꾼다.
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성하고
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 반환한다.
    tensor_tmp = tokenizer.texts_to_sequences(corpus)   

    # 토근화 15개가 넘어가면 그 문장은 제낀다.
    tensor=[]
    for i in tensor_tmp:
        if len(i) > 15:
            continue
        else:
            tensor.append(i)
    
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰준다.
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰준다
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용한다.
    # post는 뒤쪽
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor, tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2  520    3 ...    0    0    0]
 [   2   45   66 ...    0    0    0]
 [   2    9 3390 ...    0    0    0]
 ...
 [   2  561   21 ...    0    0    0]
 [   2  120   34 ...    0    0    0]
 [   2    5   22 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f4c192a6dd0>


자연어처리 분야에서 모델의 입력이 되는 문장을 소스 문장(Source Sentence) , 정답 역할을 하게 될 모델의 출력 문장을 타겟 문장(Target Sentence) 라고 관례적으로 부른다

In [70]:
# 텐서를 소스와 타겟으로 분리하여 모델이 학습할 수 있는 상태로 만든다.
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성한다.
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높다
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성한다.
tgt_input = tensor[:, 1:]

print(src_input[0])
print(tgt_input[0])

[  2 520   3   0   0   0   0   0   0   0   0   0   0   0]
[520   3   0   0   0   0   0   0   0   0   0   0   0   0]


In [71]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 12001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만든다.
# 데이터셋에 대해서는 아래 문서를 참고하세요.
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다.
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [72]:
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state=7)
print(enc_train.shape, dec_train.shape)

(124810, 14) (124810, 14)


In [73]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        # embedding_size 는 워드 벡터의 차원수, 즉 단어가 추상적으로 표현되는 크기이다.
        # 예를들어, 차갑다: [0.0, 1.0], 뜨겁다: [1.0, 0.0], 미지근하다: [0.5, 0.5]
        # LSTM 레이어의 hidden state 의 차원수인 hidden_size 도 같은 맥락이다. 
        # hidden_size 는 모델에 얼마나 많은 일꾼을 둘 것인가? 로 이해해도 크게 엇나가지 않는다.
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [74]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)

Epoch 1/10
609/609 [==============================] - 212s 341ms/step - loss: 3.9140
Epoch 2/10
609/609 [==============================] - 209s 342ms/step - loss: 3.0030
Epoch 3/10
609/609 [==============================] - 208s 341ms/step - loss: 2.8099
Epoch 4/10
609/609 [==============================] - 389s 639ms/step - loss: 2.6655
Epoch 5/10
609/609 [==============================] - 358s 588ms/step - loss: 2.5479
Epoch 6/10
609/609 [==============================] - 208s 342ms/step - loss: 2.4487
Epoch 7/10
609/609 [==============================] - 209s 342ms/step - loss: 2.3585
Epoch 8/10
609/609 [==============================] - 209s 343ms/step - loss: 2.2662
Epoch 9/10
609/609 [==============================] - 209s 343ms/step - loss: 2.1890
Epoch 10/10
609/609 [==============================] - 208s 342ms/step - loss: 2.1156


# 모델 평가하기

In [75]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력한다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아낸다
    #    3. 2.에서 예측된 word index를 문장 뒤에 붙인다.
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마친다.
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환한다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

### 가사 생성기 써보기

In [76]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you , i m a liability <end> '

# model 요약

In [77]:
model.summary()

Model: "text_generator_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      multiple                  3072256   
_________________________________________________________________
lstm_2 (LSTM)                multiple                  5246976   
_________________________________________________________________
lstm_3 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense_1 (Dense)              multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________
